In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: %s" % device)

device: cpu


# RESNet-101 Test

In [ ]:
from resnet_wrapper import ResNetWrapper

res = ResNetWrapper()
res


In [ ]:
in_t = torch.randn(1, 3, 720, 960)

out, low_level_features = res(in_t)
print(out.shape, low_level_features.shape)

# WASP Module Test



In [ ]:
from wasp import WASP
in_t = torch.randn(16, 1280, 90, 120)

w = WASP()
w

In [ ]:
out = w(in_t)
print(out.shape)

# Decoder Test

In [ ]:
from decoder import Decoder

d = Decoder()

In [ ]:
in_wasp = torch.randn(1, 256, 90, 120)
in_low_level = torch.randn(1, 256, 180, 240)

score_maps = d(in_wasp, in_low_level)
print(score_maps.shape)

# UniPose Test

In [3]:
from unipose import UniPose
u = UniPose().to(device)
u


UniPose(
  (resnet): ResNetWrapper(
    (net): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace

In [ ]:
# in_t = torch.randn(1, 3, 960, 720).to(device)

# out = u(in_t)

# print(out.shape)

In [6]:
# !conda install -c conda-forge opencv --yes
# !sudo apt-get install python-opencv -y --allow-unauthenticated
# !conda install libgtk2.0-dev --yes
import json
import numpy as np
import cv2
from google.cloud import storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pose-estimation-310415-49a5b660c254.json"

imagelist = []
kptlist = []

with open('../annotations/valid.json') as f:
    data = json.load(f)
    
storage_client = storage.Client("pose_estimation")
bucket = storage_client.get_bucket('pose_estimation_datasets')

for i in range(5):
    img_name = data[i]['image']
    
    blob = bucket.blob('MPII/images/' +  img_name)
    blob.content_type = 'image/jpeg'
    image = np.asarray(bytearray(blob.download_as_string()))
    img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)
    
    kpt = np.asarray(data[i]['joints'], dtype=np.int32)

    if img.shape[0] != 960 or img.shape[1] != 720:
        kpt[:,0] = kpt[:,0] * (960/img.shape[1])
        kpt[:,1] = kpt[:,1] * (720/img.shape[0])
        img = cv2.resize(img,(960,720))
        img = np.array(img)
    
    imagelist.append(img)
    kptlist.append(kpt)

In [7]:
import torch
torch_image = torch.Tensor(imagelist)
torch_image = torch_image.permute(0, 3, 1, 2).to(device)
print(torch_image.shape)
out = u(torch_image).to(device)

torch.Size([5, 3, 720, 960])


In [9]:
kpt_tensor = torch.FloatTensor(kptlist).to(device)
print(kpt_tensor.shape)

torch.Size([5, 16, 2])


# Naive MSE Loss Test

In [11]:
from criterion.joint_max_mse_loss import JointMaxMSELoss

# in_t = torch.randn(1, 16, 720, 960).to(device)

# correct_coords = torch.Tensor([[
#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),
# ]]).to(device)

loss = JointMaxMSELoss()

output = loss(out, kpt_tensor)

print(output)
# output.backward()


tensor(242242.8750)


# Distribution Difference Loss

In [13]:
from criterion.distribution_difference_loss import DistributionDifferenceLoss

# in_t = torch.randn(1, 16, 720, 960).to(device)

# correct_coords = np.array([[
#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),

#     (40, 40),
#     (40, 40),
#     (40, 40),
#     (40, 40),
# ]])

loss = DistributionDifferenceLoss(device)

output = loss(out, kpt_tensor)

print(output)

TypeError: rsub() received an invalid combination of arguments - got (Tensor, numpy.ndarray), but expected one of:
 * (Tensor input, Tensor other, *, Number alpha)
 * (Tensor input, Number other, Number alpha)
